In [1]:
#default_exp cli

In [2]:
#export
from nbdev.read import *
from nbdev.export import *

from nbdev.imports import *
from fastcore.script import *

from nbformat.sign import NotebookNotary
import tempfile

In [5]:
from fastcore.test import *

## Diff & trust notebooks

Before making a commit, you may want to check there is no diff between the exported library and the notebooks. You may also want to make this part of your CI, so that you don't accidentally merge a PR that introduces some changes between the two. This function is there to print this diff.

In [ ]:
#export
import subprocess
from shutil import copytree,ignore_patterns

In [ ]:
#export
@call_parse
def nbdev_diff_nbs():
    "Prints the diff between an export of the library in notebooks and the actual modules"
    lib_folder = Config().path('lib_path')
    with tempfile.TemporaryDirectory() as d1, tempfile.TemporaryDirectory() as d2:
        d1,d2 = Path(d1),Path(d2)
        path = Config().path('lib_path')
        ignore=ignore_patterns("__pycache__")
        shutil.copytree(path, d1, dirs_exist_ok=True, ignore=ignore)
        nbdev_build_lib()
        shutil.copytree(path, d2, dirs_exist_ok=True, ignore=ignore)
        shutil.rmtree(path)
        shutil.copytree(d1, str(path))
        print(run('diff', '-ru', d1, d2, ignore_ex=True)[1])

In [ ]:
# nbdev_diff_nbs()

If you receive an output, you'll need to either run `nbdev_build_lib()` or `nbdev_update_lib()` to fix the difference.

In [ ]:
#export
@call_parse
def nbdev_trust_nbs(fname:Param("A notebook name or glob to convert", str)=None,
                    force_all:Param("Trust even notebooks that haven't changed", bool)=False):
    "Trust noteboks matching `fname`"
    check_fname = Config().path('nbs_path')/".last_trusted"
    last_checked = os.path.getmtime(check_fname) if check_fname.exists() else None
    files = Config().path('nbs_path').glob('**/*.ipynb') if fname is None else glob.glob(fname)
    for fn in files:
        if force_all or not last_checked or os.path.getmtime(fn)>last_checked:
            nb = read_nb(fn)
            if not NotebookNotary().check_signature(nb): NotebookNotary().sign(nb)
    check_fname.touch(exist_ok=True)

In [ ]:
# nbdev_trust_nbs()

## Export -

In [12]:
from nbdev.doclinks import nbdev_build_lib
nbdev_build_lib()